In [11]:
from statistic_helper import *
from graph_helper import *
import json, nltk
po.offline.init_notebook_mode()

In [12]:
videos_words = load_data_from_file("videoData_vs_WordList.json")
students1 = load_data_from_file("studentBehaviorInfo_1.json")
students2 = load_data_from_file("studentBehaviorInfo_2.json")
videos_data = load_data_from_file("videoDataInfo.csv")
video_data_processed = load_data_from_file("video_data.json")
students = students1 + students2
stop_words = set(nltk.corpus.stopwords.words("english"))

In [13]:
words_frequency_video = {}
for video in videos_words:
    for word in video['wordList']:
        if word not in stop_words:
            if word in words_frequency_video:
                words_frequency_video[word] += 1
            else:
                words_frequency_video[word] = 1
video_words_sorted = sorted(words_frequency_video.items(), key=lambda item: item[1])
video_words_sorted.reverse()

In [14]:
words_frequency_vocabularylist = {}
stop_words = set(nltk.corpus.stopwords.words("english"))
for student in students:
    for word_item in student['vocabularyList']:
        word = word_item['word']
        if word not in stop_words:
            if word in words_frequency_vocabularylist:
                words_frequency_vocabularylist[word] += 1
            else:
                words_frequency_vocabularylist[word] = 1

words_vocabulary_list_sorted = sorted(words_frequency_vocabularylist.items(), key=lambda item: item[1])
words_vocabulary_list_sorted.reverse()

In [15]:
nb_inactif_user = 0
nb_user_without_chosen_video = 0
nb_user_without_vocabulary = 0
nb_user_without_score = 0
for student in students:
    if not student['chosenVideo'] and not student['vocabularyList'] and not student['listenScore']:
        nb_inactif_user += 1
    if not student['chosenVideo']:
        nb_user_without_chosen_video += 1
    if not student['vocabularyList']:
        nb_user_without_vocabulary += 1
    if not student['listenScore']:
        nb_user_without_score += 1

print "Inactif", nb_inactif_user
print "User without video", nb_user_without_chosen_video
print "User without vocabulary", nb_user_without_vocabulary
print "User without score", nb_user_without_score

Inactif 2
User without video 2
User without vocabulary 280
User without score 205


In [16]:
generate_bar_chart(video_words_sorted, 40)
generate_bar_chart(words_vocabulary_list_sorted, 40)

In [17]:
videos_word_level = []
videos_speed = []
videos_subtitles_length = []
for video in videos_data:
    videos_word_level.append(videos_data[video]['wordLevel'])
    videos_speed.append(videos_data[video]['videoSpeed'])
    videos_subtitles_length.append(videos_data[video]['subtitleLengthRatio'])

In [18]:
word_level_student = []
speed_student = []
subtitles_length_student = []
for student in students:
    average_word_level = 0.0
    average_speed = 0.0
    average_subtitles_length = 0.0
    for video in student['chosenVideo']:
        average_word_level += videos_data[video]['wordLevel']
        average_speed += videos_data[video]['videoSpeed']
        average_subtitles_length += videos_data[video]['subtitleLengthRatio']
    if student['chosenVideo']:
        nb_video = float(len(student['chosenVideo']))
        word_level_student.append(average_word_level/nb_video)
        speed_student.append(average_speed/nb_video)
        subtitles_length_student.append(average_subtitles_length/nb_video)
    

In [19]:
generate_histogram(videos_word_level, "Word difficulties", "test")
generate_histogram(word_level_student, "Word difficulties picked by the student", "test")
generate_histogram(videos_speed, "Video speed", "test")
generate_histogram(speed_student, "Video speed picked by the student", "test")
generate_histogram(videos_subtitles_length, "Subtitles length picked by the student", "test")
generate_histogram(subtitles_length_student, "Subtitles length", "test")

In [20]:
id_video = []
for video in videos_data:
    id_video.append(video)
combinations = itertools.combinations(id_video, 2)
frequency_combinations = {}
for combination in combinations:
    for student in students:
        chosen_video = student['chosenVideo']
        if len(chosen_video) >= 2:
            if combination[0] in chosen_video and combination[1] in chosen_video:
                if combination in frequency_combinations:
                    frequency_combinations[combination] += 1
                else:
                    frequency_combinations[combination] = 1

In [21]:
node_flow_user_video = []
for video in videos_data:
    node_flow_user_video.append(video)
edge_flow_user_video = []
for combination in frequency_combinations:
    nb_edge = frequency_combinations[combination] / 10
    for i in range(0, nb_edge):
        edge_flow_user_video.append(combination)
        
graph_flow_user_video = build_graph(node_flow_user_video, edge_flow_user_video)

In [22]:
edge_trace = Scatter(
    x=[], 
    y=[], 
    line=Line(width=0.1,color='#888'),
    hoverinfo='none',
    mode='lines')

node_trace = Scatter(
    x=[], 
    y=[], 
    text=[],
    mode='markers', 
    hoverinfo='text',
    marker=Marker(
        showscale=True,
        # colorscale options
        # 'Greys' | 'Greens' | 'Bluered' | 'Hot' | 'Picnic' | 'Portland' |
        # Jet' | 'RdBu' | 'Blackbody' | 'Earth' | 'Electric' | 'YIOrRd' | 'YIGnBu'
        colorscale='YIOrRd',
        reversescale=True,
        color=[], 
        size=8,         
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line=dict(width=2)))

In [23]:
node_trace_flow_user_video, edge_trace_flow_user_video = build_edge_and_node_trace(node_trace, edge_trace, graph_flow_user_video)

nodes = graph_flow_user_video.nodes()

for node, adjacencies in enumerate(graph_flow_user_video.adjacency_list()):
    node_trace_flow_user_video['marker']['color'].append(len(adjacencies))
    node_info = str(nodes[node]) + ' has ' + str(len(adjacencies)) + ' connections'
    node_trace_flow_user_video['text'].append(node_info)

fig = Figure(data=Data([edge_trace_flow_user_video, node_trace_flow_user_video]),
             layout=Layout(
                title='Flow between video',
                titlefont=dict(size=16),
                showlegend=False, 
                hovermode='closest',
                annotations=[ dict(
                    text="",
                    showarrow=False,
                    x=0.005, y=-0.002 ) ],
                xaxis=XAxis(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=YAxis(showgrid=False, zeroline=False, showticklabels=False)))

po.iplot(fig, filename='networkx')

In [24]:
distribution_similarity_stemmed = {}
distribution_similarity_original = {}
combinations = itertools.combinations(id_video, 2)

for combination in combinations:
    list_stemmed_one = [x['stemmed'] for x in video_data_processed[str(combination[0])]['wordList']] 
    list_stemmed_two = [x['stemmed'] for x in video_data_processed[str(combination[1])]['wordList']] 
    counter_stemmed_one = Counter(list_stemmed_one)
    counter_stemmed_two = Counter(list_stemmed_two)
    distribution_similarity_stemmed[combination] = counter_cosine_similarity(counter_stemmed_one, counter_stemmed_two)
    list_original_one = [x['original'] for x in video_data_processed[str(combination[0])]['wordList']] 
    list_original_two = [x['original'] for x in video_data_processed[str(combination[1])]['wordList']] 
    counter_original_one = Counter(list_original_one)
    counter_original_two = Counter(list_original_two)
    distribution_similarity_original[combination] = counter_cosine_similarity(counter_original_one, counter_original_two)
    

In [25]:
generate_histogram([value for key, value in distribution_similarity_original.iteritems()], "Distribution of the similarity (original)", "test")
generate_histogram([value for key, value in distribution_similarity_stemmed.iteritems()], "Distribution of the similarity (stemmed)", "test")

In [38]:
similarity_node = [id_v for id_v in id_video]
similarity_edge_original = []
for combination in distribution_similarity_original:
    if distribution_similarity_original[combination] > 0.20:
        similarity_edge_original.append(combination)
similarity_graph_original = build_graph(similarity_node, similarity_edge_original)

similarity_node= [id_v for id_v in id_video]
similarity_edge_stemmed = []
for combination in distribution_similarity_stemmed:
    if distribution_similarity_stemmed[combination] > 0.20:
        similarity_edge_stemmed.append(combination)
similarity_graph_stemmed = build_graph(similarity_node, similarity_edge_stemmed)

In [45]:
edge_trace = Scatter(
    x=[], 
    y=[],
    line= Line(width=0.3, color='#888'),
    hoverinfo='none',
    mode='lines'
)

node_trace = Scatter(
    x=[], 
    y=[], 
    text=[],
    mode='markers', 
    hoverinfo='text',
    marker=Marker(
        showscale=True,
        # colorscale options
        # 'Greys' | 'Greens' | 'Bluered' | 'Hot' | 'Picnic' | 'Portland' |
        # Jet' | 'RdBu' | 'Blackbody' | 'Earth' | 'Electric' | 'YIOrRd' | 'YIGnBu'
        colorscale='YIGnBu',
        reversescale=True,
        color=[], 
        size=12,         
        colorbar=dict(
            thickness=15,
            title='Similarity size',
            xanchor='left',
            titleside='right'
            ),
        line=dict(width=2)
        )
    )

In [46]:
node_trace_similarity_stemmed, edge_trace_similarity_stemmed = build_edge_and_node_trace(node_trace, edge_trace, similarity_graph_stemmed)

nodes = similarity_graph_stemmed.nodes()

for node, adjacencies in enumerate(similarity_graph_stemmed.adjacency_list()):
    node_trace_similarity_stemmed['marker']['color'].append(len(adjacencies))
    node_info = str(nodes[node]) + ' has ' + str(len(adjacencies)) + ' connections'
    node_trace_similarity_stemmed['text'].append(node_info)

In [47]:
fig_stemmed = Figure(data=Data([edge_trace_similarity_stemmed, node_trace_similarity_stemmed]),
             layout=Layout(
                title='Video similar based on the vocabulary (stemmed)',
                titlefont=dict(size=16),
                showlegend=False, 
                hovermode='closest',
                annotations=[ dict(
                    text="",
                    showarrow=False,
                    x=0.005, y=-0.002 ) ],
                xaxis=XAxis(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=YAxis(showgrid=False, zeroline=False, showticklabels=False)))

In [48]:
edge_trace = Scatter(
    x=[], 
    y=[],
    line= Line(width=0.3, color='#888'),
    hoverinfo='none',
    mode='lines'
)

node_trace = Scatter(
    x=[], 
    y=[], 
    text=[],
    mode='markers', 
    hoverinfo='text',
    marker=Marker(
        showscale=True,
        # colorscale options
        # 'Greys' | 'Greens' | 'Bluered' | 'Hot' | 'Picnic' | 'Portland' |
        # Jet' | 'RdBu' | 'Blackbody' | 'Earth' | 'Electric' | 'YIOrRd' | 'YIGnBu'
        colorscale='YIGnBu',
        reversescale=True,
        color=[], 
        size=12,         
        colorbar=dict(
            thickness=15,
            title='Similarity size',
            xanchor='left',
            titleside='right'
            ),
        line=dict(width=2)
        )
    )

In [49]:
node_trace_similarity_original, edge_trace_similarity_original = build_edge_and_node_trace(node_trace, edge_trace, similarity_graph_original)

nodes = similarity_graph_original.nodes()

for node, adjacencies in enumerate(similarity_graph_original.adjacency_list()):
    node_trace_similarity_original['marker']['color'].append(len(adjacencies))
    node_info = str(nodes[node]) + ' has ' + str(len(adjacencies)) + ' connections'
    node_trace_similarity_original['text'].append(node_info)

In [50]:
fig_original = Figure(data=Data([edge_trace_similarity_original, node_trace_similarity_original]),
             layout=Layout(
                title='Video similar based on the vocabulary (original)',
                titlefont=dict(size=16),
                showlegend=False, 
                hovermode='closest',
                annotations=[ dict(
                    text="",
                    showarrow=False,
                    x=0.005, y=-0.002 ) ],
                xaxis=XAxis(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=YAxis(showgrid=False, zeroline=False, showticklabels=False)))

In [ ]:
po.iplot(fig_original, filename='networkx')
po.iplot(fig_s, filename='networkx')